In [2]:
import random
import os
import pandas as pd
import numpy as np
from tqdm.auto import tqdm
from sklearn.preprocessing import LabelEncoder

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torch.utils.data import DataLoader, TensorDataset

In [3]:
device = torch.device('mps') if torch.cuda.is_available() else torch.device('cpu')

In [4]:
CFG = {
    'TRAIN_WINDOW_SIZE':120, # 90일치로 학습
    'PREDICT_SIZE':21, # 21일치 예측
    'EPOCHS':30,
    'LEARNING_RATE':0.0001,
    'BATCH_SIZE':512,
    'SEED':41
}

In [5]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

seed_everything(CFG['SEED']) # Seed 고정

In [6]:
train_data = pd.read_csv('/Users/leeshinhaeng/Desktop/open/train.csv').drop(columns=['ID', '제품'])

In [7]:
# Scaling 코드 (이희원님 코드 가져옴)

# 숫자형 변수들의 min-max scaling을 수행하는 코드입니다.
numeric_cols = train_data.columns[4:]
# 칵 column의 min 및 max 계산
min_values = train_data[numeric_cols].min(axis=1)
max_values = train_data[numeric_cols].max(axis=1)
# 각 행의 범위(max-min)를 계산하고, 범위가 0인 경우 1로 대체
ranges = max_values - min_values
ranges[ranges == 0] = 1
# min-max scaling 수행
train_data[numeric_cols] = (train_data[numeric_cols].subtract(min_values, axis=0)).div(ranges, axis=0)
# max와 min 값을 dictionary 형태로 저장
scale_min_dict = min_values.to_dict()
scale_max_dict = max_values.to_dict()

In [8]:
# Label Encoding
label_encoder = LabelEncoder()
categorical_columns = ['대분류', '중분류', '소분류', '브랜드']

# 범주형 칼럼별로 레이블 인코딩
for col in categorical_columns:
    label_encoder.fit(train_data[col])
    train_data[col] = label_encoder.transform(train_data[col])

In [9]:
def make_train_data(data, train_size=CFG['TRAIN_WINDOW_SIZE'], predict_size=CFG['PREDICT_SIZE']):
    '''
    학습 기간 블럭, 예측 기간 블럭의 세트로 데이터를 생성
    data : 일별 판매량
    train_size : 학습에 활용할 기간
    predict_size : 추론할 기간
    '''
    # 데이터 행 수 저장
    num_rows = len(data)
    # 전체 시간 간격 (학습 기간 + 예측 기간)
    window_size = train_size + predict_size
    
    # 빈 배열로 초기화
    input_data = np.empty((num_rows * (len(data.columns) - window_size + 1), train_size, len(data.iloc[0, :4]) + 1))
    target_data = np.empty((num_rows * (len(data.columns) - window_size + 1), predict_size))
    
    # 학습 데이터, 레이블 데이터 학습
    for i in tqdm(range(num_rows)):
        # 정보를 나타내는 4개 열을 encode_info로
        encode_info = np.array(data.iloc[i, :4])
        sales_data = np.array(data.iloc[i, 4:])
        # 학습 기간 블럭 : temp_data에 저장, input_data에 추가
        # 예측 기간 블럭 : windorw[train_size:]로 추출, target_data에 추가
        for j in range(len(sales_data) - window_size + 1):
            window = sales_data[j : j + window_size]
            temp_data = np.column_stack((np.tile(encode_info, (train_size, 1)), window[:train_size]))
            input_data[i * (len(data.columns) - window_size + 1) + j] = temp_data
            target_data[i * (len(data.columns) - window_size + 1) + j] = window[train_size:]
    
    return input_data, target_data

In [10]:
data=train_data
train_size=CFG['TRAIN_WINDOW_SIZE'] # 120
predict_size=CFG['PREDICT_SIZE'] # 21

num_rows = len(data) # 15890
window_size = train_size + predict_size # 120+21

In [11]:
def make_predict_data(data, train_size=CFG['TRAIN_WINDOW_SIZE']):
    '''
    평가 데이터(Test Dataset)를 추론하기 위한 Input 데이터를 생성
    data : 일별 판매량
    train_size : 추론을 위해 필요한 일별 판매량 기간 (= 학습에 활용할 기간)
    '''
    num_rows = len(data)
    
    # 초기 빈 배열로 초기화
    input_data = np.empty((num_rows, train_size, len(data.iloc[0, :4]) + 1))
    
    for i in tqdm(range(num_rows)):
        # 정보를 나타내는 4개 열을 encode_info로
        encode_info = np.array(data.iloc[i, :4])
        sales_data = np.array(data.iloc[i, -train_size:])
        
        window = sales_data[-train_size : ]
        temp_data = np.column_stack((np.tile(encode_info, (train_size, 1)), window[:train_size]))
        input_data[i] = temp_data
    
    return input_data

In [12]:
train_input, train_target = make_train_data(train_data)
test_input = make_predict_data(train_data)

  0%|          | 0/15890 [00:00<?, ?it/s]

  0%|          | 0/15890 [00:00<?, ?it/s]

In [13]:
# Train / Validation Split (60,20,20)
data_len = len(train_input)
val_input = train_input[-int(data_len*0.2):]
val_target = train_target[-int(data_len*0.2):]
train_input = train_input[:-int(data_len*0.2)]
train_target = train_target[:-int(data_len*0.2)]

In [14]:
train_input.shape, train_target.shape, val_input.shape, val_target.shape, test_input.shape

((4105976, 120, 5),
 (4105976, 21),
 (1026494, 120, 5),
 (1026494, 21),
 (15890, 120, 5))

In [15]:
# 주어진 데이터셋을 X와 Y로 나누어 입력과 타겟을 각각 저장
# 학습과 평가를 위한 데이터셋을 편리하게 관리
class CustomDataset(Dataset):
    def __init__(self, X, Y):
        self.X = X
        self.Y = Y
        
    def __getitem__(self, index):
        if self.Y is not None:
            return torch.Tensor(self.X[index]), torch.Tensor(self.Y[index])
        return torch.Tensor(self.X[index])
    
    def __len__(self):
        return len(self.X)

In [16]:
train_dataset = CustomDataset(train_input, train_target)
train_loader = DataLoader(train_dataset, batch_size = CFG['BATCH_SIZE'], shuffle=True, num_workers=0)

val_dataset = CustomDataset(val_input, val_target)
val_loader = DataLoader(val_dataset, batch_size = CFG['BATCH_SIZE'], shuffle=False, num_workers=0)

In [17]:
train_input.shape

(4105976, 120, 5)

In [18]:
CFG['BATCH_SIZE']

512

CFG = {
    'TRAIN_WINDOW_SIZE':120, # 90일치로 학습
    'PREDICT_SIZE':21, # 21일치 예측 -> num_layers
    'EPOCHS':30,
    'LEARNING_RATE':0.0001,
    'BATCH_SIZE':512,
    'SEED':41
}

In [146]:
# class GRU(nn.Module):
#     def __init__(self, output_size, input_size, hidden_size, num_layers, seq_length, dropout):
#         super(GRU, self).__init__()
#         self.output_size = output_size
#         self.num_layers = num_layers 
#         self.input_size = input_size 
#         self.hidden_size = hidden_size 
#         self.seq_length = seq_length
#         self.dropout = dropout

#         self.gru1 = nn.GRU(input_size=input_size, hidden_size=hidden_size,
#                             num_layers=num_layers, batch_first=True)
#         self.gru2 = nn.GRU(input_size=hidden_size, hidden_size=hidden_size,
#                             num_layers=num_layers, batch_first=True)
#         self.fc_1 = nn.Linear(hidden_size, 128)
#         self.dropout_layer = nn.Dropout(dropout)
#         self.batch_norm1 = nn.BatchNorm1d(128)
#         self.fc_2 = nn.Linear(128, 64)
#         self.fc = nn.Linear(64, output_size)  # hidden_size를 64로 변경
#         self.relu = nn.ReLU()
    
#     def forward(self, x):
#         h_01 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
#         output1, (hn1) = self.gru1(x, h_01)
#         output2, (hn2) = self.gru2(output1, hn1)
#         hn2_last_step = hn2[-1, : ,:]  # Use the last hidden state
#         return hn2_last_step  # (batch_size, hidden_size)
    
# class GRU_Transformer(nn.Module):
#     def __init__(self, input_size, hidden_size, num_layers, seq_length, dropout, nhead, num_encoder_layers, num_decoder_layers, output_size):
#         super(GRU_Transformer, self).__init__()
#         self.gru = GRU(hidden_size, input_size, hidden_size, num_layers, seq_length, dropout)
        
#         self.transformer_encoder = nn.TransformerEncoder(
#             nn.TransformerEncoderLayer(d_model=hidden_size, nhead=nhead),
#             num_layers=num_encoder_layers
#         )
        
#         self.transformer_decoder = nn.TransformerDecoder(
#             nn.TransformerDecoderLayer(d_model=hidden_size, nhead=nhead),
#             num_layers=num_decoder_layers
#         )
        
#         self.fc_out = nn.Linear(hidden_size, output_size)

#     def forward(self, x):
#         gru_out = self.gru(x)  # (batch_size, hidden_size)
        
#         encoder_out = self.transformer_encoder(gru_out.unsqueeze(0))  # Add seq_len dimension
        
#         # Assuming the decoder's target is the same as the encoder's output for auto-regression. 
#         # Otherwise, you can modify this part accordingly.
#         decoder_out = self.transformer_decoder(encoder_out, encoder_out)
        
#         # Using the last step of the decoder for prediction
#         out = self.fc_out(decoder_out[-1])
#         return out

In [ ]:
class GRU(nn.Module):
    def __init__(self, output_size, input_size, hidden_size, num_layers, seq_length, dropout):
        super(GRU, self).__init__()
        self.output_size = output_size
        self.num_layers = num_layers 
        self.input_size = input_size 
        self.hidden_size = hidden_size 
        self.seq_length = seq_length
        self.dropout = dropout

        self.gru1 = nn.GRU(input_size=input_size, hidden_size=hidden_size,
                            num_layers=num_layers, batch_first=True)
        self.gru2 = nn.GRU(input_size=hidden_size, hidden_size=hidden_size,
                            num_layers=num_layers, batch_first=True)
        self.fc_1 = nn.Linear(hidden_size, 128)
        self.dropout_layer = nn.Dropout(dropout)
        self.batch_norm1 = nn.BatchNorm1d(128)
        self.fc_2 = nn.Linear(128, 64)
        self.fc = nn.Linear(64, output_size)  # hidden_size를 64로 변경
        self.relu = nn.ReLU()
    
    def forward(self, x):
        h_01 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        output1, (hn1) = self.gru1(x, h_01)
        output2, (hn2) = self.gru2(output1, hn1)
        hn2_last_step = hn2[-1, : ,:]  # Use the last hidden state
        return hn2_last_step  # (batch_size, hidden_size)
    
class PositionalEncoding(nn.Module):
    def __init__(self, d_model, dropout=0.1, max_len=5000):
        super(PositionalEncoding, self).__init__()
        self.dropout = nn.Dropout(p=dropout)

        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0)  # <----- 이 부분을 추가하세요.
        self.register_buffer('pe', pe)

    def forward(self, x):
        x = x + self.pe[:, :x.size(1), :]
        return self.dropout(x)
    
class GRU_Transformer(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, seq_length, dropout, nhead, num_encoder_layers, num_decoder_layers, output_size):
        super(GRU_Transformer, self).__init__()
        self.seq_length = seq_length
        self.gru = GRU(hidden_size, input_size, hidden_size, num_layers, seq_length, dropout)
        
        self.positional_encoding = PositionalEncoding(hidden_size, dropout, seq_length)
        
        self.transformer_encoder = nn.TransformerEncoder(
            nn.TransformerEncoderLayer(d_model=hidden_size, nhead=nhead),
            num_layers=num_encoder_layers
        )
        
        self.transformer_decoder = nn.TransformerDecoder(
            nn.TransformerDecoderLayer(d_model=hidden_size, nhead=nhead),
            num_layers=num_decoder_layers
        )
        
        self.fc_out = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        gru_out = self.gru(x)  # (batch_size, hidden_size)
        
        # 시퀀스 차원 추가
        gru_out = gru_out.unsqueeze(1).repeat(1, self.seq_length, 1)  # (batch_size, seq_length, hidden_size)
        print("gru_out shape:", gru_out.shape)
        gru_out = self.positional_encoding(gru_out)  # Add positional encoding
        
        # 배치와 시퀀스 차원을 교환
        gru_out = gru_out.transpose(0, 1)  # (seq_length, batch_size, hidden_size)
        
        encoder_out = self.transformer_encoder(gru_out)
        
        # Assuming the decoder's target is the same as the encoder's output for auto-regression. 
        # Otherwise, you can modify this part accordingly.
        decoder_out = self.transformer_decoder(encoder_out, encoder_out)
        
        # Using the last step of the decoder for prediction
        out = self.fc_out(decoder_out[-1])
        return out

In [147]:
train_input.shape, train_target.shape

((4105976, 120, 5), (4105976, 21))

In [148]:
import math
input_size = train_input.shape[2] # 5
hidden_size = 256
output_size = 21
num_layers = 2
seq_length = train_input.shape[1]
epochs = CFG['EPOCHS'] # 30
learning_rate = CFG['LEARNING_RATE'] # 0.0001
dropout = 0.1
nhead = 8
num_encoder_layers = 2  # 임의로 설정한 값
num_decoder_layers = 2  # 임의로 설정한 값

In [149]:
# 모델, 손실 함수, 최적화 알고리즘 초기화
model = GRU_Transformer(input_size, hidden_size, num_layers, seq_length, dropout, nhead, num_encoder_layers, num_decoder_layers, output_size)
criterion = nn.MSELoss()  # 회귀 문제로 가정. 분류 문제인 경우 CrossEntropyLoss 사용
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [ ]:
from tqdm import tqdm
# Training loop with early stopping
best_val_loss = float('inf')
patience = 5  # Number of epochs to wait for improvement
early_stop_count = 0

for epoch in tqdm(range(epochs)):  # Number of epochs to train
    model.train()
    train_loss = 0.0

    for inputs, targets in tqdm(train_loader):
        optimizer.zero_grad()
        outputs = model(inputs)
        print(outputs.shape, targets.shape)
        loss = criterion(outputs, targets)
        print(loss)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()

    # Validation
    model.eval()
    val_loss = 0.0

    with torch.no_grad():
        for inputs, targets in val_loader:
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            print(loss)
            val_loss += loss.item()

    train_loss /= len(train_loader)
    val_loss /= len(val_loader)

    print(f'Epoch [{epoch+1}/30], Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}')

    # Check for early stopping
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        early_stop_count = 0
    else:
        early_stop_count += 1
        if early_stop_count >= patience:
            print("Early stopping! No improvement in validation loss.")
            break

  0%|                                                  | 0/8020 [00:00<?, ?it/s]

gru_out shape: torch.Size([512, 120, 256])
torch.Size([512, 21]) torch.Size([512, 21])
tensor(0.2405, grad_fn=<MseLossBackward>)


In [ ]:
test_dataset = CustomDataset(test_input, None)
test_loader = DataLoader(test_dataset, batch_size = CFG['BATCH_SIZE'], shuffle=False, num_workers=0)

In [ ]:
def inference(model, test_loader):
    predictions = []

    with torch.no_grad():
        # gradient 계산하지 않도록
        for X in tqdm(iter(test_loader)):
            # 입력 데이터 전달
            output = model(X)

            # 모델 출력인 output을 CPU로 이동하고 numpy 배열로 변환
            output = output.cpu().numpy()

            # (batch_size, 1, output_size)에서 (batch_size, output_size)로 변경
#             output = np.squeeze(output, axis=1)
            print(output.shape)
            predictions.extend(output)

    return np.array(predictions)

In [63]:
# 학습이 완료된 모델을 평가 모드로 전환
model.eval()

# test_loader를 사용하여 예측 수행
predictions = inference(model, test_loader)

  0%|                                                    | 0/32 [00:00<?, ?it/s]

h_01 :  torch.Size([2, 512, 256])
output1 :  torch.Size([512, 120, 256])
hn1 :  torch.Size([2, 512, 256])


  3%|█▍                                          | 1/32 [00:04<02:28,  4.80s/it]

output2 :  torch.Size([512, 120, 256])
hn2 :  torch.Size([2, 512, 256])
hn2_last_step :  torch.Size([512, 120, 256])
out1 :  torch.Size([512, 128])
out2 :  torch.Size([512, 64])
out3 :  torch.Size([512, 21])
torch.Size([512, 21])
(512, 21)
h_01 :  torch.Size([2, 512, 256])
output1 :  torch.Size([512, 120, 256])
hn1 :  torch.Size([2, 512, 256])


  6%|██▊                                         | 2/32 [00:09<02:25,  4.85s/it]

...

output2 :  torch.Size([512, 120, 256])
hn2 :  torch.Size([2, 512, 256])
hn2_last_step :  torch.Size([512, 120, 256])
out1 :  torch.Size([512, 128])
out2 :  torch.Size([512, 64])
out3 :  torch.Size([512, 21])
torch.Size([512, 21])
(512, 21)
h_01 :  torch.Size([2, 512, 256])
output1 :  torch.Size([512, 120, 256])
hn1 :  torch.Size([2, 512, 256])


 97%|█████████████████████████████████████████▋ | 31/32 [02:29<00:04,  4.89s/it]

output2 :  torch.Size([512, 120, 256])
hn2 :  torch.Size([2, 512, 256])
hn2_last_step :  torch.Size([512, 120, 256])
out1 :  torch.Size([512, 128])
out2 :  torch.Size([512, 64])
out3 :  torch.Size([512, 21])
torch.Size([512, 21])
(512, 21)
h_01 :  torch.Size([2, 18, 256])
output1 :  torch.Size([18, 120, 256])
hn1 :  torch.Size([2, 18, 256])


100%|███████████████████████████████████████████| 32/32 [02:29<00:00,  4.68s/it]

output2 :  torch.Size([18, 120, 256])
hn2 :  torch.Size([2, 18, 256])
hn2_last_step :  torch.Size([18, 120, 256])
out1 :  torch.Size([18, 128])
out2 :  torch.Size([18, 64])
out3 :  torch.Size([18, 21])
torch.Size([18, 21])
(18, 21)


In [65]:
predictions.shape

(15890, 21)

In [66]:
# 추론 결과를 inverse scaling
for idx in range(len(predictions)):
    predictions[idx, :] = predictions[idx, :] * (scale_max_dict[idx] - scale_min_dict[idx]) + scale_min_dict[idx]

# 결과 후처리
pred = np.round(predictions, 0).astype(int)

In [67]:
pred.shape

(15890, 21)

In [68]:
pred

array([[ 2,  1,  0, ..., -1,  0,  0],
       [ 1,  1,  0, ...,  0,  0,  0],
       [ 5,  3, -2, ..., -2, -1,  0],
       ...,
       [ 4,  2, -1, ..., -2, -1,  0],
       [ 2,  1, -1, ..., -1, -1,  0],
       [ 0,  0,  0, ...,  0,  0,  0]])